In [2]:
import pandas as pd

In [99]:
root_table = 'E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/crawl_table'

In [122]:
def extract_quarter(text):
    """Hàm chuẩn hóa cách viết quý, bán niên, năm"""
    mapping = {
        "quý 1": "Quý I", "quý i/": "Quý I",
        "quý 2": "Quý II", "quý ii/": "Quý II",
        "quý 3": "Quý III", "quý iii/": "Quý III",
        "quý 4": "Quý IV", "quý iv/": "Quý IV",
        "bán niên": "Bán niên",
        "năm": "Năm"
    }
    text = text.lower().strip()
    for key, val in mapping.items():
        if key in text:
            return val
    return None


def extract_type(text):
    """Hàm chuẩn hóa cách viết quý, bán niên, năm"""
    mapping = {
        "riêng": "Riêng", "Riêng": "Riêng",
        "tổng hợp": "Tổng hợp", "Tổng hợp": "Tổng hợp",
        "Mẹ": "Riêng", "mẹ": "Riêng",
        "hợp nhất": "Tổng hợp", "Hợp nhất": "Tổng hợp"
    }
    text = text.lower().strip()
    for key, val in mapping.items():
        if key in text:
            return val
    return None

def map_source(df_before, df_after):
    # 1️⃣ Trích xuất year & range từ "Tên báo cáo" của df_before
    df_before["year"] = df_before["Tên báo cáo"].str.extract(r'(\d{4})').astype(str)
    df_after["year"] = df_after["year"].astype(str).str.extract(r'(\d{4})')

    df_before["range"] = df_before["Tên báo cáo"].apply(extract_quarter)

    # Trích xuất "type" từ "Tên báo cáo"
    df_before["type"] = df_before["Tên báo cáo"].apply(extract_type).fillna("Riêng")

    # 3️⃣ Tạo key chung để merge
    df_before["key"] = df_before["year"] + "_" + df_before["range"] + "_" + df_before["type"]
    df_after["key"] = df_after["year"] + "_" + df_after["range"] + "_" + df_after["type"]

    df_merged = df_before.merge(df_after, on="key", how="left")

    print("Nb source report: ", len(df_merged))
    print("Nb check report: ", len(df_after))

    df_merged = df_merged.loc[~df_merged["tax_code"].isna()]
    
    print("Nb mapped report: ", len(df_merged))

    return df_merged


In [123]:
df_after = pd.read_csv("test_kqkt.csv")
df_after = df_after.loc[df_after["stock_code"] == "HHG"]

df_before = pd.read_csv(root_table + "/HHG/success.csv")

In [124]:
df_merged = map_source(df_before, df_after)

Nb source report:  74
Nb check report:  21
Nb mapped report:  21


In [125]:
df_merged['Saving_path'].to_list()[0]

'./crawl_table/HHG/Báo cáo tài chính quý 2_ 2024_24_07_2024 00_00_00'

In [126]:
df_merged

,Unnamed: 0,STT,Tên báo cáo,Đơn vị,Trích yếu,Thời gian gửi,Mã doanh nghiệp,Tên công ty,Tiêu đề,Saving_path,...,Lợi nhuận sau thuế_21_LK0,Lợi nhuận sau thuế_21_LK1,Lợi ích của cổ đông thiểu số_22_0,Lợi ích của cổ đông thiểu số_22_1,Lợi ích của cổ đông thiểu số_22_LK0,Lợi ích của cổ đông thiểu số_22_LK1,Lãi cơ bản trên cổ phiếu_23_0,Lãi cơ bản trên cổ phiếu_23_1,Lãi cơ bản trên cổ phiếu_23_LK0,Lãi cơ bản trên cổ phiếu_23_LK1
3,3,4,Báo cáo tài chính quý 2/ 2024,CTCP Hoàng Hà,,24/07/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính quý 2/ 2024,./crawl_table/HHG/Báo cáo tài chính quý 2_ 202...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,Báo cáo tài chính quý 2/ 2024 của công ty mẹ,CTCP Hoàng Hà,,24/07/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính quý 2/ 2024 của công ty mẹ,./crawl_table/HHG/Báo cáo tài chính quý 2_ 202...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,10,Báo cáo tài chính Mẹ - Quý 4/ 2023,CTCP Hoàng Hà,Báo cáo tài chính Mẹ - Quý 4/2023,20/01/2024 16:34:05,1000272301,CTCP Hoàng Hà,Báo cáo tài chính Mẹ - Quý 4/ 2023,./crawl_table/HHG/Báo cáo tài chính Mẹ - Quý 4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,11,Báo cáo tài chính hợp nhất quý 4/ năm 2023,CTCP Hoàng Hà,Báo cáo tài chính Hợp nhất - Quý 4/2023,20/01/2024 16:27:50,1000272301,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất quý 4/ năm 2023,./crawl_table/HHG/Báo cáo tài chính Hợp nhất -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,12,Báo cáo tài chính hợp nhất quý 3/ năm 2023,CTCP Hoàng Hà,Báo cáo tài chính Hợp nhất - Quý 3/2023,19/10/2023 10:44:32,1000272301,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất quý 3/ năm 2023,./crawl_table/HHG/Báo cáo tài chính Hợp nhất -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,16,Báo cáo tài chính hợp nhất quý 2/ năm 2023,CTCP Hoàng Hà,Báo cáo tài chính Hợp nhất - Quý 2/2023,20/07/2023 15:43:01,1000272301,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất quý 2/ năm 2023,./crawl_table/HHG/Báo cáo tài chính Hợp nhất -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,17,18,Báo cáo tài chính hợp nhất quý 1/ năm 2023,CTCP Hoàng Hà,Báo cáo tài chính Hợp nhất - Quý 1,28/04/2023 14:38:51,1000272301,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất quý 1/ năm 2023,./crawl_table/HHG/Báo cáo tài chính Hợp nhất -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,23,24,Báo cáo tài chính hợp nhất quý 3/ năm 2022,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất tại ngày 30 tháng 0...,19/10/2022 16:19:41,1000272301,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất quý 3/ năm 2022,./crawl_table/HHG/Báo cáo tài chính hợp nhất q...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,26,27,Báo cáo tài chính Mẹ - Bán niên năm 2022,CTCP Hoàng Hà,Báo cáo tài chính riêng giữa niên độ đã được s...,22/08/2022 15:16:46,1000272301,CTCP Hoàng Hà,Báo cáo tài chính Mẹ - Bán niên năm 2022,./crawl_table/HHG/Báo cáo tài chính Mẹ - Bán n...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,30,Báo cáo tài chính hợp nhất quý 1/ năm 2022,CTCP Hoàng Hà,Báo cáo tài chính Hợp nhất - Quý I/2022,20/04/2022 15:55:18,1000272301,CTCP Hoàng Hà,Báo cáo tài chính hợp nhất quý 1/ năm 2022,./crawl_table/HHG/Báo cáo tài chính Hợp nhất -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
root_source = "E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/"

In [133]:
pd.read_csv(root_source + "/crawl_table/HHG/Báo cáo tài chính quý 2_ 2024_24_07_2024 00_00_00/BCDKT.csv")

,Chỉ tiêu,Mã số,Thuyết minh,Số cuối kỳ,Số đầu kỳ
0,TÀI SẢN,NaN,NaN,NaN,NaN
1,A- TÀI SẢN NGẮN HẠN,100,NaN,"4,705,633,183","7,116,287,657"
2,I. Tiền và các khoản tương đương tiền,110,NaN,"743,605,703","281,614,602"
3,1. Tiền,111,NaN,"743,605,703","281,614,602"
4,2. Các khoản tương đương tiền,112,NaN,0,0
...,...,...,...,...,...
114,13. Lợi ích cổ đông không kiểm soát,429,NaN,0,0
115,II. Nguồn kinh phí và quỹ khác,430,NaN,0,0
116,1. Nguồn kinh phí,431,NaN,0,0
117,2. Nguồn kinh phí đã hình thành TSCĐ,432,NaN,0,0


In [140]:
pd.read_csv(root_source + "/crawl_table/HHG/Báo cáo tài chính quý 2_ 2024_24_07_2024 00_00_00/KQKT.csv").iloc[:, 0].astype(str).str.contains("TÀI SẢN NGẮN HẠN", na=False).any()

True

In [136]:
pd.read_csv(root_source + "/crawl_table/HHG/Báo cáo tài chính quý 2_ 2024_24_07_2024 00_00_00/LCTT_TT1.csv")

,Chỉ tiêu,Mã số,Thuyết minh,Lũy kế từ đầu năm đến cuối kỳ này Năm nay,Lũy kế từ đầu năm đến cuối kỳ này Năm trước
0,I. Lưu chuyển tiền từ hoạt động kinh doanh,NaN,NaN,NaN,NaN
1,"1. Tiền thu từ bán hàng, cung cấp dịch vụ và d...",1.00,NaN,0.0,0.0
2,2. Tiền chi trả cho người cung cấp hàng hóa và...,2.00,NaN,0.0,0.0
3,3. Tiền chi trả cho người lao động,3.00,NaN,0.0,0.0
4,4. Tiền lãi vay đã trả,4.00,NaN,0.0,0.0
5,5. Thuế thu nhập doanh nghiệp đã nộp,5.00,NaN,0.0,0.0
6,6. Tiền thu khác từ hoạt động kinh doanh,6.00,NaN,0.0,0.0
7,7. Tiền chi khác cho hoạt động kinh doanh,7.00,NaN,0.0,0.0
8,Lưu chuyển tiền thuần từ hoạt động kinh doanh,20.00,NaN,0.0,0.0
9,II. Lưu chuyển tiền từ hoạt động đầu tư,NaN,NaN,NaN,NaN


In [139]:
pd.read_csv(root_source + "/crawl_table/HHG/Báo cáo tài chính quý 2_ 2024_24_07_2024 00_00_00/LCTT_TT2.csv")

,Chỉ tiêu,Mã số,Thuyết minh,Lũy kế từ đầu năm đến cuối kỳ này Năm nay,Lũy kế từ đầu năm đến cuối kỳ này Năm trước
0,I. Lưu chuyển tiền từ hoạt động kinh doanh,NaN,NaN,NaN,NaN
1,1. Lợi nhuận trước thuế,1.0,NaN,"-17,736,921,128","-23,470,920,118"
2,2. Điều chỉnh cho các khoản,NaN,NaN,NaN,NaN
3,- Khấu hao TSCĐ và BĐSĐT,2.0,NaN,"10,035,151,205","10,675,574,244"
4,- Các khoản dự phòng,3.0,NaN,"-122,756,312",0
5,"- Lãi, lỗ chênh lệch tỷ giá hối đoái do đánh g...",4.0,NaN,"-143,863","-7,272"
6,"- Lãi, lỗ từ hoạt động đầu tư",5.0,NaN,"-134,010,977","-27,264,603"
7,- Chi phí lãi vay,6.0,NaN,"1,052,629,841","2,830,884,697"
8,- Các khoản điều chỉnh khác,7.0,NaN,0,0
9,3. Lợi nhuận từ hoạt động kinh doanh trước tha...,8.0,NaN,"-6,906,051,234","-9,991,733,052"


In [ ]:
"crawl_table"

In [150]:
import os
import pandas as pd
from tqdm import tqdm

# Đường dẫn đến thư mục chứa các stockid
base_folder = "E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/crawl_table"
source = "E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/crawl_table"

# Biến đếm
total_files = 0
error_files = 0

# Duyệt qua các thư mục stockid
for stockid in tqdm(os.listdir(base_folder)):
    stock_folder = os.path.join(base_folder, stockid)
    
    if not os.path.isdir(stock_folder):
        continue  # Bỏ qua nếu không phải thư mục

    # Đường dẫn đến file success.csv
    success_path = os.path.join(stock_folder, "success.csv")
    
    if not os.path.exists(success_path):
        # print(f"{stockid} khong co success.csv")
        continue  # Bỏ qua nếu không có success.csv

    try:
        df_success = pd.read_csv(success_path)
        # print(df_success)
        
        # Duyệt qua từng saving_path trong success.csv
        for saving_path in df_success["Saving_path"].dropna():
            saving_path = saving_path[13:]
            # print(source + saving_path)
            kqkt_path = os.path.join(source + saving_path, "KQKT.csv")
            
            if not os.path.exists(kqkt_path):
                continue  # Bỏ qua nếu không có file KQKT.csv
            
            total_files += 1

            try:
                df_kqkt = pd.read_csv(kqkt_path, encoding="utf-8", header=None)
                
                # Kiểm tra xem "TÀI SẢN NGẮN HẠN" có trong cột đầu tiên không
                if df_kqkt.iloc[:, 0].astype(str).str.contains("TÀI SẢN NGẮN HẠN", na=False).any():
                    error_files += 1  # Tăng số lần sai

            except Exception as e:
                print(f"Lỗi đọc file {kqkt_path}: {e}")

    except Exception as e:
        print(f"Lỗi đọc file {success_path}: {e}")

# Tính tỷ lệ %
error_rate = (error_files / total_files) * 100 if total_files > 0 else 0

print(f"Tổng số file: {total_files}")
print(f"Số file bị lỗi: {error_files}")
print(f"Tỷ lệ lỗi: {error_rate:.2f}%")


100%|██████████| 1525/1525 [04:34<00:00,  5.56it/s]

Tổng số file: 68930
Số file bị lỗi: 41536
Tỷ lệ lỗi: 60.26%


In [152]:
import os
import pandas as pd
from tqdm import tqdm

# Đường dẫn đến thư mục chứa các stockid
base_folder = "E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/crawl_table"
source = "E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/crawl_table"

# Biến đếm
total_files = 0
error_files = 0

# Duyệt qua các thư mục stockid
for stockid in tqdm(os.listdir(base_folder)):
    stock_folder = os.path.join(base_folder, stockid)
    
    if not os.path.isdir(stock_folder):
        continue  # Bỏ qua nếu không phải thư mục

    # Đường dẫn đến file success.csv
    success_path = os.path.join(stock_folder, "success.csv")
    
    if not os.path.exists(success_path):
        # print(f"{stockid} khong co success.csv")
        continue  # Bỏ qua nếu không có success.csv

    try:
        df_success = pd.read_csv(success_path)
        # print(df_success)
        
        # Duyệt qua từng saving_path trong success.csv
        for saving_path in df_success["Saving_path"].dropna():
            saving_path = saving_path[13:]
            # print(source + saving_path)
            kqkt_path = os.path.join(source + saving_path, "LCTT_TT2.csv")
            
            if not os.path.exists(kqkt_path):
                continue  # Bỏ qua nếu không có file KQKT.csv
            
            total_files += 1

            try:
                df_kqkt = pd.read_csv(kqkt_path, encoding="utf-8", header=None)
                
                # Kiểm tra xem "TÀI SẢN NGẮN HẠN" có trong cột đầu tiên không
                if df_kqkt.iloc[:, 0].astype(str).str.contains("TÀI SẢN NGẮN HẠN", na=False).any():
                    error_files += 1  # Tăng số lần sai

            except Exception as e:
                print(f"Lỗi đọc file {kqkt_path}: {e}")

    except Exception as e:
        print(f"Lỗi đọc file {success_path}: {e}")

# Tính tỷ lệ %
error_rate = (error_files / total_files) * 100 if total_files > 0 else 0

print(f"Tổng số file: {total_files}")
print(f"Số file bị lỗi: {error_files}")
print(f"Tỷ lệ lỗi: {error_rate:.2f}%")


100%|██████████| 1525/1525 [03:35<00:00,  7.09it/s]

Tổng số file: 68925
Số file bị lỗi: 12
Tỷ lệ lỗi: 0.02%
